# DADA2 re-run to check rarefaction curves for  Exp 1

* DADA2 pairwise sample inference remves singletons, which makes rarefaction curves plataue at any sequencing depth.  This notebook re-runs the sample inference Exp 1 to check sequencing depth of various sample types.

* Need to define for each dataset
** Number of cores
** OutFolder
** Path to sequence directory

In [ ]:
ncores = 20

In [ ]:
library(dada2)
library(tidyr)
library(dplyr)

In [ ]:
seqDIR = '~/Hyphosphere/data/MiSeq/Exp1/'
path = seqDIR

#location of taxonomy database
#TrainingSet = '~/databases/silva_nr_v132_train_set.fa.gz'
#SpeciesTraining = '~/databases/silva_species_assignment_v132.fa.gz'

# Sample data table
SamTab = '~/Hyphosphere/3exp_metadata_final.txt'

# output folder
OutFolder = '~/Hyphosphere/data/Exp1/DADA2Files/Pooled-rerun'


# Filter and trim


In [ ]:
#Collect forward and reverse names in order
fastqFs <- sort(list.files(path, pattern=".R1_001.fastq"))
fastqRs <- sort(list.files(path, pattern=".R2_001.fastq"))
if(length(fastqFs) != length(fastqRs)) stop("Forward and reverse files do not match.")

In [ ]:
#sample.names

## Plot quality profiles of forward and reverse reads

In [ ]:
plotQualityProfile(file.path(path,fastqFs[17:18]))


* Foward reads look very good.  Truncate to 220

In [ ]:
plotQualityProfile(file.path(path,fastqRs[17:18]))

# Perform filtering and trimming


In [ ]:
filtpathF <- file.path(path, "filtered_F") # Filtered forward files go into the pathF/filtered/ subdirectory
filtpathR <- file.path(path, "filtered_R") # ...

* Note that primers remain on sequences from UMGC, so trimLeft(19,20) is included in this dataset

In [ ]:
# Filtering: THESE PARAMETERS ARENT OPTIMAL FOR ALL DATASETS
out = filterAndTrim(fwd=file.path(path, fastqFs), filt=file.path(filtpathF, fastqFs),
              rev=file.path(path, fastqRs), filt.rev=file.path(filtpathR, fastqRs),
              truncLen=c(210,125), maxEE=2, truncQ=2, maxN=0, rm.phix=TRUE, trimLeft=c(19,20),
              compress=TRUE, verbose=TRUE, multithread=ncores)

In [ ]:
head(out)

In [ ]:
filtFs <- list.files(filtpathF, pattern=".fastq", full.names = TRUE)
filtRs <- list.files(filtpathR, pattern=".fastq", full.names = TRUE)
sample.names <- sapply(strsplit(basename(filtFs), "_R1_001.fastq"), `[`, 1) # Assumes filename = samplename_XXX.fastq.gz
sample.namesR <- sapply(strsplit(basename(filtRs), "_R2_001.fastq"), `[`, 1) # Assumes filename = samplename_XXX.fastq.gz
if(!identical(sample.names, sample.namesR)) stop("Forward and reverse files do not match.")
names(filtFs) <- sample.names
names(filtRs) <- sample.names
sample.names

## Learn errors

In [ ]:
set.seed(100)
# Learn forward error rates
errF <- learnErrors(filtFs, multithread=ncores)
# Learn reverse error rates
errR <- learnErrors(filtRs, multithread=ncores)

In [ ]:
plotErrors(errF)

## Sequence inference

In [ ]:
# # Sample inference and merger of paired-end reads
# mergers <- vector("list", length(sample.names))
# names(mergers) <- sample.names
# for(sam in sample.names) {
#   cat("Processing:", sam, "\n")
#     derepF <- derepFastq(filtFs[[sam]])
#     ddF <- dada(derepF, err=errF, multithread=ncores, pool = TRUE)
#     derepR <- derepFastq(filtRs[[sam]])
#     ddR <- dada(derepR, err=errR, multithread=ncores, pool = TRUE)
#     merger <- mergePairs(ddF, derepF, ddR, derepR)
#     mergers[[sam]] <- merger
# }
# rm(derepF); rm(derepR)

In [ ]:
derepF <- derepFastq(filtFs)
    dadaFS <- dada(derepF, err=errF, multithread=ncores, pool = TRUE)

In [ ]:
derepR <- derepFastq(filtRs)
    dadaRS <- dada(derepR, err=errR, multithread=ncores, pool = TRUE)

In [ ]:
mergers <- mergePairs(dadaFS, derepF, dadaRS, derepR, verbose=TRUE)
# Inspect the merger data.frame from the first sample
head(mergers[[1]])

# Construct sequence table and remove chimeras

In [ ]:

seqtab <- makeSequenceTable(mergers)

In [ ]:
(seqtab)[1:3,1:5]

In [ ]:
outFile = file.path(OutFolder,"seqtab_wPooledDadaPipeline.rds")
saveRDS(seqtab, outFile) 

In [ ]:
dim(seqtab)
# Inspect distribution of sequence lengths
table(nchar(getSequences(seqtab)))

## subset to proper sequence lengths

In [ ]:
seqtab2 <- seqtab[,nchar(colnames(seqtab)) %in% seq(251,256)]

table(nchar(getSequences(seqtab2)))

## Remove chimeras

* removes all sequences that can be exactly reconstructed as a bimera of two more abundant sequences

In [ ]:
seqtab.nochim <- removeBimeraDenovo(seqtab2, method="consensus", multithread=ncores, verbose=TRUE)

In [ ]:
dim(seqtab.nochim)

In [ ]:
outFile = file.path(OutFolder,"seqtab-seqtab_wPooledDadaPipeline.rds")

saveRDS(seqtab.nochim, outFile)

# Generate rarefaction curves

In [ ]:
out = vegan::rarecurve(seqtab.nochim,step = 50, cex = 0.5)

In [ ]:
rare <- lapply(out, function(x){
  b <- as.data.frame(x)
  b <- data.frame(OTU = b[,1], raw.read = rownames(b))
  b$raw.read <- as.numeric(gsub("N", "",  b$raw.read))
  return(b)
})

In [ ]:
names(rare) <- rownames(seqtab.nochim)

In [ ]:
library(tidyverse)

In [ ]:
rare <- map_dfr(rare, function(x){
  z <- data.frame(x)
  return(z)
}, .id = "sample")
head(rare)

In [ ]:
ggplot(data = rare)+
  geom_line(aes(x = raw.read, y = OTU, color = sample))+
  scale_x_continuous(labels =  scales::scientific_format())

In [ ]:
df = read.delim(SamTab, sep = '\t', header = TRUE)
head(df)

In [ ]:
rare = left_join(rare, df, by = c("sample"="SampleID"))

In [ ]:
head(rare)

In [ ]:
theme_pub = function (base_size = 11, base_family = "Helvetica") 
{
    theme_grey(base_size = base_size, base_family = base_family) %+replace% 
        theme(
            axis.line.x = element_line(size = .5),
            axis.line.y = element_line(size = .5),
            panel.background = element_rect(fill = "white", colour = NA), 
            panel.border = element_rect(fill = NA, color = "black", size = .5), 
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank(), 
            strip.background = element_rect(fill = "grey85", 
                colour = "grey20"), 
            legend.key = element_rect(fill = "white", 
                colour = NA), complete = TRUE)
    }

In [ ]:
cs = c('#D95F02', '#7570B3')
plot = rare %>%
filter(SampleType %in% c("CH", "CS")) %>%
ggplot()+
  geom_line(aes(x = raw.read, y = OTU, group = sample, color = SampleType))+
  scale_x_continuous(labels =  scales::scientific_format())+
  scale_color_manual(values = cs, labels = c("Core ERH", "Soil slurry"))+
  facet_wrap(~Soil, ncol = 1, scales = 'free_y')+
  theme_pub()+
  xlab("Raw read")+
  ylab("Observed ASVs")+
  labs(color = "Sample type")

plot

In [ ]:
pdf(file = '~/Hyphosphere/data/3Exp/figures/final/RarefactionCurves.pdf', width = 5, height = 7, useDingbats = FALSE)
plot
dev.off()